In [1]:
from keras.models import Sequential
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense, Activation, Dropout

np.random.seed(42)

train = pd.read_csv('train.csv.zip', compression='zip')
X_train = train.drop(['id', 'species'], axis=1).values
le = LabelEncoder().fit(train['species'])
Y_train = le.transform(train['species'])

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

model = Sequential()
model.add(Dense(1024,input_dim=192, init='uniform', activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(99, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.fit(X_train, Y_train, nb_epoch=32, batch_size=32, validation_split=0.33)

test = pd.read_csv('test.csv.zip', compression='zip')
test_ids = test.pop('id')
X_test = test.values
X_test = scaler.transform(X_test)

Y_test = model.predict_proba(X_test)

submission = pd.DataFrame(Y_test, index=test_ids, columns=le.classes_)
submission.to_csv('submission_nn.csv')

Using TensorFlow backend.
/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Train on 663 samples, validate on 327 samples
Epoch 1/32
663/663 [==============================] - 0s - loss: 3.8639 - acc: 0.2293 - val_loss: 2.9486 - val_acc: 0.4924
Epoch 2/32
663/663 [==============================] - 0s - loss: 2.2043 - acc: 0.5958 - val_loss: 1.9978 - val_acc: 0.6942
Epoch 3/32
663/663 [==============================] - 0s - loss: 1.4087 - acc: 0.8100 - val_loss: 1.3406 - val_acc: 0.8104
Epoch 4/32
663/663 [==============================] - 0s - loss: 0.8852 - acc: 0.8974 - val_loss: 0.8356 - val_acc: 0.9327
Epoch 5/32
663/663 [==============================] - 0s - loss: 0.5468 - acc: 0.9457 - val_loss: 0.5327 - val_acc: 0.9664
Epoch 6/32
663/663 [==============================] - 0s - loss: 0.3233 - acc: 0.9789 - val_loss: 0.3647 - val_acc: 0.9602
Epoch 7/32
663/663 [==============================] - 0s - loss: 0.2182 - acc: 0.9804 - val_loss: 0.2442 - val_acc: 0.9817
Epoch 8/32
663/663 [==============================] - 0s - loss: 0.1373 - acc: 0.9879 - val_l

In [2]:
# 663/663 [==============================] - 0s - loss: 3.1089e-05 - acc: 1.0000 - val_loss: 0.0157 - val_acc: 0.9939


In [6]:
from sklearn.model_selection import StratifiedKFold
seed = 7
np.random.seed(seed)

# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X_train, Y_train):
  # create model
    model = Sequential()
    model.add(Dense(1024,input_dim=192, init='uniform', activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='sigmoid'))
    model.add(Dropout(0.3))
    model.add(Dense(99, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    # Fit the model
    model.fit(X_train, Y_train, nb_epoch=50, batch_size=10, verbose=True)
    # evaluate the model
    #scores = model.evaluate(X_test, Y_test, verbose=True)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    #cvscores.append(scores[1] * 100)



Epoch 1/50
990/990 [==============================] - 1s - loss: 3.1298 - acc: 0.3717     
Epoch 2/50
990/990 [==============================] - 0s - loss: 0.9141 - acc: 0.8515     
Epoch 3/50
990/990 [==============================] - 0s - loss: 0.2934 - acc: 0.9626     
Epoch 4/50
990/990 [==============================] - 0s - loss: 0.1207 - acc: 0.9848     
Epoch 5/50
990/990 [==============================] - 0s - loss: 0.0545 - acc: 0.9939     
Epoch 6/50
990/990 [==============================] - 0s - loss: 0.0286 - acc: 0.9949     
Epoch 7/50
990/990 [==============================] - 0s - loss: 0.0181 - acc: 0.9949     
Epoch 8/50
990/990 [==============================] - 0s - loss: 0.0088 - acc: 0.9980     
Epoch 9/50
990/990 [==============================] - 0s - loss: 0.0078 - acc: 0.9970     
Epoch 10/50
990/990 [==============================] - 0s - loss: 0.0074 - acc: 0.9970     
Epoch 11/50
990/990 [==============================] - 0s - loss: 0.0017 - acc: 1.0000   

KeyboardInterrupt: 

In [5]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

test = pd.read_csv('test.csv.zip', compression='zip')
test_ids = test.pop('id')
X_test = test.values
X_test = scaler.transform(X_test)

Y_test = model.predict_proba(X_test)

submission = pd.DataFrame(Y_test, index=test_ids, columns=le.classes_)
submission.to_csv('submission_nn_2.csv')

nan% (+/- nan%)
594/594 [==============================] - 0s     

/opt/conda/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/opt/conda/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.5/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/opt/conda/lib/python3.5/site-packages/numpy/core/_methods.py:94: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/opt/conda/lib/python3.5/site-packages/numpy/core/_methods.py:116: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
